In [5]:
import numpy as np
import pandas as pd 
import bluepy
from pathlib import Path
import logging
from tqdm import tqdm
from voxcell import VoxelData
from voxcell.nexus.voxelbrain import Atlas
from shutil import copyfile

def generate_nrrd_from_target(target_name,circuit_dir):
    ''' Creates nrrd mask from the target in the bluepy Curcuit but only for the voxels belonging to gids. Not a cylinder mask'''
    #CIRCUIT_DIR = Path('/gpfs/bbp.cscs.ch/project/proj112/scratch/circuits/20211026-BioM_test4/')
    c = bluepy.Circuit((circuit_dir / 'CircuitConfig_struct').as_posix())
    df = c.cells.get(target_name)
    atlas = Atlas.open(ATLAS_DIR.as_posix())
    orientation = atlas.load_data("orientation")
    mask_indices = orientation.positions_to_indices(df[['x','y','z']].values)
    data = np.full(orientation.shape, False) # make an array of Falses with the shape of the orientation
    for i in mask_indices: # change indices to True by looking into mask_indices
        data[i[0],i[1],i[2]] = True
    generated_mask = orientation.with_data(np.asarray(data, dtype=np.uint8)).save_nrrd(target_name+'.nrrd')  # pick the smallest integer type
    return generated_mask

def get_voxel_positions(atlas_dir):
    '''Loads an nrrd file and returns a pandas df with indices as voxel coordinates and positions as values'''
    atlas = Atlas.open(atlas_dir)
    orientation = atlas.load_data("orientation")
    ndindex = list(np.ndindex(orientation.shape))
    pos_per_voxel = orientation.indices_to_positions(ndindex)
    return pos_per_voxel

def mask_voxels_in_cylinder(of_radius, in_atlas, centered_at, with_axis_along):
    """..."""
    voxel_positions = get_voxel_positions(in_atlas)
    c = np.cross(voxel_positions - centered_at, with_axis_along)
    distances = np.linalg.norm(c,axis=1) / np.linalg.norm(with_axis_along)
    return distances <= of_radius

def get_gids_from_mask(mask_name,circuit_path):
    c = bluepy.Circuit(circuit_path + '/CircuitConfig_struct')
    atlas = Atlas.open('.')
    mask = atlas.load_data(mask_name)
    in_target = mask.lookup(c.cells.get(properties=list("xyz")).values)
    gids_in_target = c.cells.get().index.values[np.where(in_target==1)] #assuming mask is 0/1
    return gids_in_target


In [6]:
def update_user_target(filename, gids, target_name, notes=''):
    with open(filename, 'a') as f:
        f.write(notes)
        f.write('\n')
        f.write('Target Cell %s\n{\n' % target_name)
        f.write(' '.join(['a%d' % gid for gid in gids]))
        f.write('\n}\n\n')

In [7]:
def check_cylinder(df,target_name,low_threshold=2400,high_threshold=6000):
    cylinder_df = df[df[target_name] == True]
    #gids = get_gids_in_voxel(mask,circuit) #
    total = 0
    mtypes = circuit.cells.mtypes
    for mtype in mtypes:
        ncells = len(cylinder_df[cylinder_df["mtype"]==mtype])
        total += ncells
        #print(mtype, ncells)
        if ncells < 1:
            print(f'Validatian failed. {mtype} is not in the circuit.')
            return False
    if total < low_threshold:
        print(f'Validatian failed. There are fewer cells than expected ({total})')
        return False
    return True

In [8]:
def get_points_in_cylinder(point_id, distance, df, target_name):
    point_row = df.loc[point_id]
    point = np.array([point_row['x'], point_row['y'], point_row['z']])
    vector_line = p.loc[point_id]['orientation'][:,1]
    def is_in_cylinder(row):
        candidate = np.array([row['x'], row['y'], row['z']])
        return distance > distance_to_cylinder(point, vector_line, candidate)
    df[target_name] = df.apply(is_in_cylinder, axis=1)

In [9]:
def generate_cylinder_mask(circuit,reference_gid,cylinder_radius,mask_name):
    # GENERATE GEOMETRIC MASK FROM TARGET CELL (no gaps in between)
    # Look for the all voxel's positions and get the indices of the ones closer than 300 um from the center of cylinder.
    cells =  circuit.cells.get()
    point_row = cells.loc[reference_gid]
    center_point = np.array([point_row['x'], point_row['y'], point_row['z']])
    center_vector_line = cells.loc[reference_gid]['orientation'][:,1]
    masked_cylinder = mask_voxels_in_cylinder(cylinder_radius,ATLAS_DIR.as_posix(),center_point,center_vector_line)
    masked_cylinder_reshaped = masked_cylinder.reshape(orientation.shape)
    return masked_cylinder_reshaped

In [10]:
def get_gids_in_voxel(mask,circuit):
    in_target = mask.lookup(circuit.cells.get(properties=list("xyz")).values)
    gids_in_target = circuit.cells.get().index.values[np.where(in_target==1)]
    return gids_in_target

In [11]:
def check_cylinder_voxel_mask(mask,circuit,low_threshold=2400):
    temp = orientation.with_data(np.asarray(mask, dtype=np.uint8))
    temp_gids = get_gids_in_voxel(temp,circuit)
    mtypes = circuit.cells.mtypes
    temp_cylinder = circuit.cells.get(temp_gids)
    total = 0
    
    for mtype in mtypes:
        ncells = len(temp_cylinder[temp_cylinder["mtype"]==mtype])
        total += ncells
        #print(mtype, ncells)
        if ncells < 1:
            print(f'Validatian failed. {mtype} is not in the circuit.')
            return False
    if total < low_threshold:
        print(f'Validatian failed. There are fewer cells than expected ({total})')
        return False
    return True

In [12]:
np.random.seed(123)

In [13]:
CIRCUIT_DIR = Path('/gpfs/bbp.cscs.ch/project/proj112/circuits/CA1/20211110-BioM/')
ATLAS_DIR = Path("/gpfs/bbp.cscs.ch/project/proj112/entities/atlas/20211004_BioM/")
#reference_gid = 145562
target_name = 'cylinder300'
region_name = 'CA1'
cylinder_radius = 150

c = bluepy.Circuit((CIRCUIT_DIR / 'CircuitConfig_struct').as_posix())
atlas = Atlas.open(ATLAS_DIR.as_posix())
orientation = atlas.load_data("orientation")

# If central gid is not selected yet

In [14]:
from coordinate_query import CoordinateQuery, enriched_cells_positions, query_enriched_positions, LON, TRA, RAD

file_path = ATLAS_DIR / 'coordinates.nrrd'
p = c.cells.get(properties=['mtype', 'x', 'y', 'z', 'orientation'])
q = CoordinateQuery(file_path.as_posix())
xyz = enriched_cells_positions(c, q)


# this will query gids with l1 < Long < l2, t1 < Transverse < t2 and r1 < radial < r2
#gids = query_enriched_positions(xyz, l1, l2, 0.48, 0.52, 0., 1.0)
#reference_gid = np.random.choice(gids)
#reference_gid

In [30]:
xyz = xyz.sort_values(by = ['l','t','r'])

In [64]:
np.count_nonzero(xyz.l.value_counts().values > 1)

125686

In [65]:
len(xyz.l.value_counts().values > 1)

267184

In [ ]:
l1_range = np.arange(0,1,0.01)
mid_range_low = 0
mid_range_high = 1
threshold_drop = 10
l_interval = 0.001
for idx,l1 in enumerate(l1_range):
    l2 = l1+l_interval

    # this will query gids with l1 < Long < l2, t1 < Transverse < t2 and r1 < radial < r2
    gids = query_enriched_positions(xyz, l1, l2, mid_range_low, mid_range_high, 0., 1.0)
    np.random.shuffle(gids)
    print(f'There are {len(gids)} candidate ref gids for cylinder no {idx+1}')
    
    for i in range(threshold_drop):
        reference_gid = gids[i]
        target_name = 'cylinder' + str(reference_gid) + 'r' + str(cylinder_radius)
        print('Trying ref gid:',target_name)
        #get_points_in_cylinder(reference_gid, radius, p, target_name)
        cylinder_mask = generate_cylinder_mask(c,reference_gid,cylinder_radius,target_name)
        # copy nrrd to atlas dir for later steps or load data from here
        
        if check_cylinder_voxel_mask(cylinder_mask,c,low_threshold=600):
            orientation.with_data(np.asarray(cylinder_mask, dtype=np.uint8)).save_nrrd(target_name+'.nrrd')
            int_mask = Atlas.open('.').load_data(target_name)
            mask_gids = get_gids_in_voxel(int_mask,c)
            notes = '# reference_gid = ' + str(reference_gid) + ', radius = ' + str(cylinder_radius) + ', longitudinal position = ' + str(l1)
            #update_user_target(CIRCUIT_DIR.as_posix() + '/user.target', mask_gids, target_name, notes)
            update_user_target('tile.target', mask_gids, target_name, notes)
            break
        else:
            del p[target_name]
        if i==threshold_drop-1:
            print(f'Couldnt find a target between {l1}-{l2}.',end='\n\n')
    
    # Check if the cylinder is roughly perpendicular to the CA1 and is centered along the transverse axis
    # If everything looks good, you can proceed to copt the target into the circuit
    



In [ ]:
!grep 'cylinder' user.target

In [ ]:
import re

In [ ]:
cylinders = []
with open("user.target","r") as file:
    for line in file:
        if re.search('cylinder', line):
            cylinders.append(line.split()[2])
cylinders

In [ ]:
def add_cylinders_to_target(filename, gids, target_name, notes=''):
    with open(filename, 'a') as f:
        f.write(notes)
        f.write('\n')
        f.write('Target Cell %s\n{\n' % target_name)
        f.write(' '.join(['a%d' % gid for gid in gids]))
        f.write('\n}\n\n')

In [ ]:
target_str = ''
for i in cylinders:
    target_str += i
    target_str += ' '
target_str

In [ ]:
with open('user.target','a') as f:
    f.write('Target Cell %s\n{\n' % 'cylinders')
    f.write(' '.join(['%s' % c for c in cylinders]))
    f.write('\n}\n\n')

In [ ]:
copyfile(target_name+'.nrrd', ATLAS_DIR / ('cylinders' + target_name + '.nrrd'))

# Generate gids from voxel mask

In [ ]:
int_mask = Atlas.open('.').load_data(target_name)
mask = np.asarray(int_mask.raw, dtype=bool)
#cylinder300_gids = list(c.cells.get(target_name).index.values)
ca1_mask = atlas.get_region_mask('CA1').raw
intersection_mask = np.logical_and(mask,ca1_mask) # raw
intersection_mask_raw = np.asarray(intersection_mask, dtype=bool)
intersection_mask = orientation.with_data(np.asarray(intersection_mask_raw, dtype=np.uint8)) # get voxeldata format

In [ ]:
intersection_mask.save_nrrd(target_name+'_intersection_ca1.nrrd')

In [ ]:
#positions = c.cells.get(target_name, list("xyz"))
in_target = intersection_mask.lookup(c.cells.get(properties=list("xyz")).values)
not_in_target = np.logical_not(in_target)
gids_in_target = c.cells.get().index.values[np.where(in_target==1)]
gids_not_in_target = c.cells.get().index.values[np.where(not_in_target==True)]

In [ ]:
gids_in_target

In [ ]:
def get_gids_in_voxel(mask,circuit):
    in_target = mask.lookup(circuit.cells.get(properties=list("xyz")).values)
    gids_in_target = circuit.cells.get().index.values[np.where(in_target==1)]
    return gids_in_target

In [ ]:
mask_gids = get_gids_in_voxel(int_mask,c)
intersection_gids = get_gids_in_voxel(intersection_mask,c)

In [ ]:
mask_gids

In [ ]:
np.array_equal(mask_gids,intersection_gids)

In [ ]:
l1 = 0.25
notes = '# reference_gid = ' + str(reference_gid) + ', radius = ' + str(cylinder_radius) + ', longitudinal position = ' + str(l1)
update_user_target(CIRCUIT_DIR.as_posix() + '/user.target', mask_gids, target_name, notes)

## Check if gids in cylinder300 are the same in test4 and 20211110

In [ ]:
CIRCUIT_DIR2 = Path('/gpfs/bbp.cscs.ch/project/proj112/scratch/circuits/20211026-BioM_test4/')
c2 = bluepy.Circuit((CIRCUIT_DIR2 / 'CircuitConfig_struct').as_posix())

In [ ]:
# reload circuit again
np.array_equal(c.cells.ids('cylinder300'),c2.cells.ids('cylinder300'))